In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import copy  # deepcopy를 위해 추가

# ---------------------- #
# 기본 변수 초기화 (입력 값)
# ---------------------- #
initial_price = 0.0001  # 초기 가격 (유지)
slope = 0.000001  # 기울기 소폭 감소
weight_log_multiplier = 32.5  # 가중치 약간 감소
weight_log_denominator = 100000000  # 가중치 로그 내 분모 (유지)

supply_exponent = 3.0  # 발행량의 제곱근 (증가)
eth_price_usd = 4000  # 현재 ETH 가격 ($)

# ---------------------- #
# 기본 변수 초기화 (고정 값)
# ---------------------- #
initial_spread = 0.9  # 초기 스프레드
final_spread = 0.1  # 최종 스프레드
eth_per_unit = 100  # 단위별 유입 ETH 양
max_supply = 100_000_000  # 최대 발행량 목표

# ---------------------- #
# 1. 본딩 커브 계산 함수
# ---------------------- #
def calculate_bonding_curve(initial_price, slope, supply_exponent, weight_log_multiplier, 
                            weight_log_denominator, max_supply, eth_per_unit, initial_spread, final_spread):
    """
    본딩 커브와 관련된 값들을 계산하는 함수
    """
    dynamic_cumulative_supply = 0
    current_price = initial_price
    dynamic_n = 0
    weight = 0

    # 최대 발행량 도달 시점 기록 변수
    max_supply_reached_n = None

    # 목표 발행량에 도달할 때까지 n 계산
    while dynamic_cumulative_supply < max_supply:
        weight = weight_log_multiplier * np.log1p(dynamic_cumulative_supply / weight_log_denominator)
        current_price = initial_price + slope * weight * (dynamic_cumulative_supply ** (1 / supply_exponent))
        issued_supply = eth_per_unit / current_price
        dynamic_cumulative_supply += issued_supply

        # 최대 발행량에 도달하면 기록
        if dynamic_cumulative_supply >= max_supply and max_supply_reached_n is None:
            max_supply_reached_n = dynamic_n
        dynamic_n += 1

    spread_decrement = (initial_spread - final_spread) / max_supply_reached_n

    # 테이블 생성
    data = []
    cumulative_supply = 0
    spread = None
    current_price = initial_price
    sell_price = None
    weight = None
    area_buy_eth = 0
    area_sell_eth = 0

    for n in range(max_supply_reached_n + 1):
        if n == 0:
            spread = initial_spread
            data.append({
                "Unit (n)": n,
                "ETH Input (Cumulative)": 0,
                "Issued Tokens": 0,
                "Cumulative Supply": cumulative_supply,
                "Buy Price (ETH)": current_price,
                "Spread": None,
                "Sell Price (ETH)": None,
                "Weight": None
            })
            continue

        weight = weight_log_multiplier * np.log1p(cumulative_supply / weight_log_denominator)
        current_price = initial_price + slope * weight * (cumulative_supply ** (1 / supply_exponent))
        eth_input = eth_per_unit
        issued_supply = eth_input / current_price
        prev_cumulative_supply = cumulative_supply
        cumulative_supply += issued_supply
        spread = max(final_spread, spread - spread_decrement)
        sell_price = current_price * (1 - spread)

        supply_diff = cumulative_supply - prev_cumulative_supply
        area_buy_eth += current_price * supply_diff
        area_sell_eth += sell_price * supply_diff

        data.append({
            "Unit (n)": n,
            "ETH Input (Cumulative)": eth_input * n,
            "Issued Tokens": issued_supply,
            "Cumulative Supply": cumulative_supply,
            "Buy Price (ETH)": current_price,
            "Spread": spread,
            "Sell Price (ETH)": sell_price,
            "Weight": weight
        })

    area_buy_usd = area_buy_eth * eth_price_usd
    area_sell_usd = area_sell_eth * eth_price_usd

    return data, max_supply_reached_n, area_buy_eth, area_sell_eth, area_buy_usd, area_sell_usd

# ---------------------- #
# 2. 본딩 커브 시뮬레이션 실행
# ---------------------- #

# 계산 함수 실행
data, max_supply_reached_n, area_buy_eth, area_sell_eth, area_buy_usd, area_sell_usd = calculate_bonding_curve(
    initial_price, slope, supply_exponent, weight_log_multiplier, weight_log_denominator, 
    max_supply, eth_per_unit, initial_spread, final_spread)

# 테이블 생성
df = pd.DataFrame(data)
df["Buy Price (ETH)"] = df["Buy Price (ETH)"].map("{:.6e}".format)
df["Sell Price (ETH)"] = df["Sell Price (ETH)"].map("{:.6e}".format)

# ---------------------- #
# 3. 테이블 기반 계산 함수
# ---------------------- #

def find_price_at_eth(data, target_eth):
    """
    테이블에서 특정 누적 ETH 유입 시점에 해당하는 가격을 찾는 함수.
    """
    for row in data:
        if row["ETH Input (Cumulative)"] >= target_eth:
            return row["Buy Price (ETH)"], row["Unit (n)"]
    return None, None  # 찾지 못한 경우

# ---------------------- #
# 4. 분석 결과 계산
# ---------------------- #

# 목표 누적 ETH 입력값 변경
target_eth = 10000  # 10,000 ETH 유입 시점으로 변경

# 10,000 ETH 유입 시점의 가격과 해당 n값 찾기
price_at_10k_eth, n_at_10k_eth = find_price_at_eth(data, target_eth)

# 최대 발행량 시점의 가격(P_max)
price_at_max_supply = data[-1]["Buy Price (ETH)"]

# 초기 가격 대비 10,000 ETH 유입 시 가격 변화율
if price_at_10k_eth:
    price_increase_10k_eth = ((price_at_10k_eth - initial_price) / initial_price) * 100
else:
    price_increase_10k_eth = None

# ---------------------- #
# 5. 분석 결과 출력
# ---------------------- #

# 출력
print("\n### Bonding Curve Analysis ###")
print(f"Initial Price: {initial_price:.6f} ETH")
print(f"Slope: {slope}")
print(f"Weight Log Multiplier: {weight_log_multiplier}")
print(f"Weight Log Denominator: {weight_log_denominator}")
print(f"Total Buy Area (ETH): {area_buy_eth:.2f} ETH")  # Buy Area 출력
print(f"Price at 0 ETH (Initial Price): {initial_price:.6f} ETH")
if price_at_10k_eth:
    print(f"Price at 10,000 ETH Input: {price_at_10k_eth:.6f} ETH (at n = {n_at_10k_eth})")
    print(f"Price Increase (Initial to 10,000 ETH): {price_increase_10k_eth:.2f}%")
else:
    print("Price at 10,000 ETH Input: Not Found")
    print("Price Increase (Initial to 10,000 ETH): Not Available")
print(f"Price at Max Supply: {price_at_max_supply:.6f} ETH")
if price_at_10k_eth:
    print(f"Price Ratio (Max Supply / 10,000 ETH): {price_at_max_supply / price_at_10k_eth:.2f}x")
else:
    print("Price Ratio: Not Available")


### Bonding Curve Analysis ###
Initial Price: 0.000100 ETH
Slope: 1e-06
Weight Log Multiplier: 32.5
Weight Log Denominator: 100000000
Total Buy Area (ETH): 503200.00 ETH
Price at 0 ETH (Initial Price): 0.000100 ETH
Price at 10,000 ETH Input: 0.001309 ETH (at n = 100)
Price Increase (Initial to 10,000 ETH): 1209.10%
Price at Max Supply: 0.010554 ETH
Price Ratio (Max Supply / 10,000 ETH): 8.06x
